In [1]:
from google.colab import drive

In [2]:
 drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import keras

from keras.preprocessing.image import  ImageDataGenerator, img_to_array, load_img

from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions

In [4]:
#EDA

len(os.listdir("/content/drive/MyDrive/data/dataset/train"))

15

In [5]:
train_datagen= ImageDataGenerator(zoom_range=0.5,shear_range= 0.3 ,horizontal_flip=True,preprocessing_function= preprocess_input)

val_datagen=ImageDataGenerator(preprocessing_function= preprocess_input)

test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

In [6]:
train=train_datagen.flow_from_directory(directory="/content/drive/MyDrive/data/dataset/train",target_size=(256,256),batch_size=32)

val=val_datagen.flow_from_directory(directory="/content/drive/MyDrive/data/dataset/validation",target_size=(256,256),batch_size=32)

test=test_datagen.flow_from_directory(directory="/content/drive/MyDrive/data/dataset/test",target_size=(256,256),batch_size=32)

Found 16504 images belonging to 15 classes.
Found 2070 images belonging to 15 classes.
Found 2064 images belonging to 15 classes.


In [7]:
t_img,label = train.next()

In [8]:
def plotImage(img_arr,label):

  for im , l in zip(img_arr , label):
    plt.figure(figsize=(5,5))

    plt.show()

In [9]:
plotImage(t_img[:3],label[:3])

<Figure size 500x500 with 0 Axes>

<Figure size 500x500 with 0 Axes>

<Figure size 500x500 with 0 Axes>

# **BUILDING** **OUR** **MODEL**

In [10]:
from keras.layers import Dense,Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
import keras

In [11]:
base_model = VGG19(input_shape=(256,256,3), include_top= False)


80134624/80134624 [==============================] - 3s 0us/step


In [12]:
for layer in base_model.layers:
  layer.trainable = False

In [13]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [14]:
x= Flatten()(base_model.output)

x=Dense(units =15 ,activation='softmax')(x)



#creating our model
model= Model(base_model.input,x)


In [15]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [16]:
model.compile(optimizer= 'adam'  , loss = keras.losses.categorical_crossentropy , metrics =['accuracy'])

# **EARLY STOPPING AND MODEL CHECK POINT**

In [17]:
from keras.callbacks import ModelCheckpoint ,EarlyStopping

#early stopping
es = EarlyStopping(monitor= 'val_accuracy', min_delta= 0.01 ,patience = 3, verbose= 1)


#model check point
mc = ModelCheckpoint(filepath="best_model.h5",
                     monitor= 'val_accuracy',
                     min_delta= 0.01 ,
                     patience = 3,
                     verbose= 1,
                     save_best_only= True)



cb=[es,mc]


In [ ]:
his = model.fit_generator(train ,
                          steps_per_epoch =16,
                          epochs= 50 ,
                          verbose = 1,
                          callbacks=cb ,
                          validation_data= val ,
                          validation_steps= 16 )

<ipython-input-18-1f75b53fb020>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  his = model.fit_generator(train ,


Epoch 1/50
16/16 [==============================] - ETA: 0s - loss: 16.8457 - accuracy: 0.3301 
Epoch 1: val_accuracy improved from -inf to 0.59375, saving model to best_model.h5
16/16 [==============================] - 1150s 73s/step - loss: 16.8457 - accuracy: 0.3301 - val_loss: 8.3786 - val_accuracy: 0.5938
Epoch 2/50


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


16/16 [==============================] - ETA: 0s - loss: 10.6548 - accuracy: 0.5312 
Epoch 2: val_accuracy improved from 0.59375 to 0.61328, saving model to best_model.h5
16/16 [==============================] - 1138s 72s/step - loss: 10.6548 - accuracy: 0.5312 - val_loss: 9.3232 - val_accuracy: 0.6133
Epoch 3/50
16/16 [==============================] - ETA: 0s - loss: 8.6101 - accuracy: 0.5859 
Epoch 3: val_accuracy improved from 0.61328 to 0.71094, saving model to best_model.h5
16/16 [==============================] - 1134s 72s/step - loss: 8.6101 - accuracy: 0.5859 - val_loss: 5.7371 - val_accuracy: 0.7109
Epoch 4/50
 8/16 [==============>...............] - ETA: 4:37 - loss: 7.0653 - accuracy: 0.6641

In [ ]:
h = his.history
h.keys()



In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'] , c="red")
plt.title("acc vs v-acc")
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'] , c="red")
plt.title("loss vs v-loss")
plt.show()

In [ ]:
#load best_model

from keras.models import load_model

model= load_model("/content/best_model.h5")

In [ ]:
acc = model.evaluate_generator(val)[1]


print(f"the accuracy of your model is {acc*100} %")

In [ ]:
ref = dict(zip(list(train.class_indices.values()),list(train.class_indices.keys())))

In [ ]:
def  prediction(path):

  img = load_img(path, target_size=(256,256))

  i = img_to_array(img)

  im = preprocess_input(i)


  img = np.expand_dims(im , axis=0)

  pred = np.argmax(model.predict(img) )

  print(f" the image belongs to { ref[pred]}")



In [ ]:
path =  "/content/drive/MyDrive/data/dataset/test/Potato___Early_blight/009c8c31-f22d-4ffd-8f16-189c6f06c577___RS_Early.B 7885.JPG"

prediction(path)